In [1]:
import pandas as pd
import sqlite3
from src.Benchmarking.BenchmarkingService import BenchmarkingService
from src.Benchmarking.Benchmarking import Benchmarking
from src.PairsTrading.PairsTradingEnvironment import PairsTradingEnvironment
from stable_baselines3 import DQN, PPO
from src.Benchmarking.BacktestingEnvironment import BacktestingEnvironment



## Load Data 


In [2]:
pair = 0
query = f'SELECT * FROM test_pairs WHERE pair = {pair} ORDER BY Date'
with sqlite3.connect('src/Data/pairs_trading.db') as conn:
    test = pd.read_sql(query, conn)
query = f'SELECT * FROM training_pairs WHERE pair = {pair} ORDER BY Date'
with sqlite3.connect('src/Data/pairs_trading.db') as conn:
    train = pd.read_sql(query, conn).drop(columns='pair')
network = f'src/Output/nets/net_{pair}.pkl'
stats = f'src/Output/stats/stats_{pair}.pkl'

In [3]:
train_env = PairsTradingEnvironment(train, 100000)
model_non_linear = DQN('MlpPolicy', train_env, learning_starts=1000, verbose=1)
model_non_linear.learn(total_timesteps=20000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/Users/PJA/anaconda3/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


----------------------------------
| rollout/            |          |
|    ep_len_mean      | 873      |
|    ep_rew_mean      | -0.0427  |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 4        |
|    fps              | 122      |
|    time_elapsed     | 28       |
|    total_timesteps  | 3492     |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 7.16e-05 |
|    n_updates        | 622      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 873      |
|    ep_rew_mean      | 0.329    |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 8        |
|    fps              | 113      |
|    time_elapsed     | 61       |
|    total_timesteps  | 6984     |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 6.14e-05 |
|    n_updates      

In [3]:
#model_non_linear.save("model_non_linear")

model_non_linear = DQN.load("src/Benchmarking/trained_benchmark_models/model_0.zip")

In [15]:
test.tail()

,z_score,z_score_MA_50,z_score_MA_25,z_score_MA_10,z_score_MA_5,z_score_MA_50_current,z_score_MA_25_current,z_score_MA_10_current,z_score_MA_5_current,z_score_lagged,...,macd,macd_signal,macd_signal_difference,spread,Date,price_asset_1,price_asset_2,weight_1,weight_2,pair
2544,0.547915,0.428244,0.626159,0.617384,0.546870,0.765968,0.610045,0.027616,0.115824,0.501011,...,0.460904,0.496896,0.533909,1.278471,2023-05-25 00:00:00,98.3975,92.9737,0.539978,0.460022,0
2545,0.551655,0.458762,0.638028,0.647763,0.563971,0.765548,0.609917,0.027546,0.115642,0.547915,...,0.478719,0.518045,0.538493,1.284018,2023-05-26 00:00:00,98.4073,92.9364,0.539967,0.460033,0
2546,0.557521,0.481886,0.653581,0.678897,0.578545,0.765336,0.609781,0.027494,0.115539,0.551655,...,0.492575,0.538733,0.535745,1.290893,2023-05-29 00:00:00,98.4073,92.8775,0.539956,0.460044,0
2547,0.494691,0.495778,0.669682,0.692933,0.580481,0.764626,0.608582,0.027293,0.114770,0.557521,...,0.480125,0.551897,0.490918,1.264546,2023-05-30 00:00:00,98.4114,93.0884,0.539949,0.460051,0
2548,0.480637,0.515499,0.682582,0.699031,0.575665,0.764381,0.608254,0.027246,0.114641,0.494691,...,0.463572,0.557926,0.448406,1.259846,2023-05-31 00:00:00,98.4046,93.1156,0.539944,0.460056,0


In [4]:
env_non_linear = BacktestingEnvironment(test.copy(), 1000000)
obs = env_non_linear.reset()
done = False
while not done:
    action, _states = model_non_linear.predict(obs)
    print(action)
    obs, rewards, done, info = env_non_linear.step(action)
    print(env_non_linear.portfolio.portfolio_value)

2
CHECK 0
1000000


In [7]:
env_non_linear.portfolio.get_portfolio_info().tail()

,0,1,2,3,4,5
0,0,0,0,0,0,0


In [11]:
import gymnasium as gym

from stable_baselines3 import DQN

env = gym.make("CartPole-v1", render_mode="human")

model = DQN("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000, log_interval=4)
model.save("dqn_cartpole")

del model # remove to demonstrate saving and loading

model = DQN.load("dqn_cartpole")

obs, info = env.reset()
while True:
    action, _states = model.predict(obs, deterministic=True)
    print(action)
    obs, reward, terminated, truncated, info = env.step(action)
    if terminated or truncated:
        obs, info = env.reset()

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


KeyboardInterrupt: 

In [15]:
env_non_linear.portfolio.get_portfolio_info().portfolio_value.plot()

AttributeError: 'DataFrame' object has no attribute 'portfolio_value'